In [ ]:
import argparse
import logging
from pathlib import Path
from typing import Literal, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import (
    calculate_weights,
    get_available_trackers,
    load_trackers,
    get_rebalance_dates,
    cap_long_only_weights,
    calc_covariance,
    cov_to_vols,
)
from backtest import backtest2
from entities import FX_TRACKER_DICT, EM_CDS_TRACKER_DICT

from bwlogger import StyleAdapter, basic_setup
from bwutils import open_file
from portfolio.construction import calculate_weights as calculate_weights_fh

# testing

# Plot

In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)
list_series = []

In [ ]:
file_paths = list(FOLDER.glob("*.xlsx"))

In [ ]:
FOLDER = Path(
    "C:/Users/pcampos/OneDrive - Insper - Instituto de Ensino e Pesquisa/Dissertação Mestrado/Analysis/backtests"
)


list_series = []


for file_path in FOLDER.glob("*.xlsx"):


    s_backtest = pd.read_excel(file_path, index_col=0)["backtest"]


    s_backtest.name = file_path.stem

    list_series.append(s_backtest)


df_backtest = pd.concat(list_series, axis=1)
df_backtest

In [ ]:
OUTPUT_FOLDER = Path(r"C:\Users\pcampos\OneDrive - Insper - Instituto de Ensino e Pesquisa\Dissertação Mestrado\Analysis")

data = np.log(df_backtest.dropna(how="all")).diff(1).cumsum()
title = "Teste"
columns = data.columns[:10]


def plot_trackers_returns(data, title, columns):
    plt.rcParams["font.family"] = "Calibri"
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(5, 3))

    sns.lineplot(data=data[columns], palette="tab10", linewidth=1, dashes=False)


    plt.legend(loc="center right", bbox_to_anchor=(0, 0.5), fontsize=8, frameon=False)

    plt.gca().yaxis.set_ticks_position("right")
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"  {y:.00%}"))

    plt.grid(True, linestyle="--", linewidth=0.7)

    plt.xlabel(None)
    plt.ylabel("Log-Retorno", fontsize=8)
    plt.gca().yaxis.set_label_position("right")


    plt.xlim(data.index.min(), data.index.max())

    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)
    plt.tick_params("both", length=0)

    plt.title(title, fontsize=10, fontweight="bold")

    plt.savefig(OUTPUT_FOLDER.joinpath(f"{title}.svg"), bbox_inches="tight")
    plt.show()

In [ ]:
dict_groups = {
    
}
for group in dict_groups:
    plot_trackers_returns(data[group], group, columns)

In [100]:
{
    "Long Only": ["L-CDS-EW", "L-CDS-IV", "L-FX-EW", "L-FX-IV"],
    "Long Short Ativo": [
        "L-CDS-S-FX-EW",
        "L-CDS-S-FX-IV-BN",
        "L-CDS-S-FX-IV",
        "L-FX-S-CDS-EW",
        "L-FX-S-CDS-IV-BN",
        "L-FX-S-CDS-IV",
    ],
    "Long Short Pares": [
        "LS-CDS_FX-BN-BRL",
        "LS-CDS_FX-IV-BRL",
        "LS-CDS_FX-BN-CLP",
        "LS-CDS_FX-IV-CLP",
        "LS-CDS_FX-BN-CNY",
        "LS-CDS_FX-IV-CNY",
        "LS-CDS_FX-BN-COP",
        "LS-CDS_FX-IV-COP",
        "LS-CDS_FX-BN-IDR",
        "LS-CDS_FX-IV-IDR",
        "LS-CDS_FX-BN-MXN",
        "LS-CDS_FX-IV-MXN",
        "LS-CDS_FX-BN-RUB",
        "LS-CDS_FX-IV-RUB",
        "LS-CDS_FX-BN-TRY",
        "LS-CDS_FX-IV-TRY",
        "LS-CDS_FX-BN-ZAR",
        "LS-CDS_FX-IV-ZAR",
    ],
    "Long Short Classe": [
        "LS-CDS-FX-BN-IV",
        "LS-CDS-FX-IV",
        "LS-FX-CDS-BN-IV",
        "LS-FX-CDS-IV",
    ],
    "Timeseries Momentum": [
        "TSMOM-3",
        "TSMOM-6",
        "TSMOM-12",
        "TSMOM-CDS-12",
        "TSMOM-CDS-3",
        "TSMOM-CDS-6",
        "TSMOM-FX-12",
        "TSMOM-FX-3",
        "TSMOM-FX-6",
    ],
    "Valor": [
        "VALUE-CDS-PAIRED",
        "VALUE-FX-PAIRED",
    ],
    "Cross-Sectional Momentum": [
        "XSMOM-CDS",
        "XSMOM-FX",
        "XSMOM",
    ],
}

{'Long Only': ['L-CDS-EW', 'L-CDS-IV', 'L-FX-EW', 'L-FX-IV'],
 'Long Short Ativo': ['L-CDS-S-FX-EW',
  'L-CDS-S-FX-IV-BN',
  'L-CDS-S-FX-IV',
  'L-FX-S-CDS-EW',
  'L-FX-S-CDS-IV-BN',
  'L-FX-S-CDS-IV'],
 'Long Short Pares': ['LS-CDS_FX-BN-BRL',
  'LS-CDS_FX-IV-BRL',
  'LS-CDS_FX-BN-CLP',
  'LS-CDS_FX-IV-CLP',
  'LS-CDS_FX-BN-CNY',
  'LS-CDS_FX-IV-CNY',
  'LS-CDS_FX-BN-COP',
  'LS-CDS_FX-IV-COP',
  'LS-CDS_FX-BN-IDR',
  'LS-CDS_FX-IV-IDR',
  'LS-CDS_FX-BN-MXN',
  'LS-CDS_FX-IV-MXN',
  'LS-CDS_FX-BN-RUB',
  'LS-CDS_FX-IV-RUB',
  'LS-CDS_FX-BN-TRY',
  'LS-CDS_FX-IV-TRY',
  'LS-CDS_FX-BN-ZAR',
  'LS-CDS_FX-IV-ZAR'],
 'Long Short Classe': ['LS-CDS-FX-BN-IV',
  'LS-CDS-FX-IV',
  'LS-FX-CDS-BN-IV',
  'LS-FX-CDS-IV'],
 'Timeseries Momentum': ['TSMOM-3',
  'TSMOM-6',
  'TSMOM-12',
  'TSMOM-CDS-12',
  'TSMOM-CDS-3',
  'TSMOM-CDS-6',
  'TSMOM-FX-12',
  'TSMOM-FX-3',
  'TSMOM-FX-6'],
 'Valor': ['VALUE-CDS-PAIRED', 'VALUE-FX-PAIRED'],
 'Cross-Sectional Momentum': ['XSMOM-CDS', 'XSMOM-FX', 'XSMOM']}